## Build Time - Tool Test Case Generation Example

Demonstrates using Tool Test Case Generation from ALTK to generate test case values and NL Utterances that formulates the test case values into a user query for validating robustness of tools and agents.

#### Python Tool:

In [1]:
import os

os.environ["WX_API_KEY"] = "xxxx"
os.environ["WX_PROJECT_ID"] = "xxxx"

In [2]:
python_tool = '''
from enum import Enum
from typing import ClassVar, Optional, Type
import datetime
from dataclasses import dataclass
from langchain_core.tools import tool

class TimeOffTypes(Enum):
    """Represents the time off event types"""

    ABSENCE = "ABSENCE"
    PUBLIC_HOLIDAY = "PUBLIC_HOLIDAY"
    NON_WORKING_DAY = "NON_WORKING_DAY"

@dataclass
class UpcomingTimeOff:
    """Represents an upcoming time off event"""

    title: str
    start_date: str
    end_date: str
    start_time: Optional[str]
    end_time: Optional[str]
    duration: int
    time_unit: str
    cross_midnight: bool
    type: str
    status_formatted: Optional[str]
    absence_duration_category: Optional[str]

    #Schema: ClassVar[Type[Schema]] = Schema

@dataclass
class UpcomingTimeOffResponse:
    """Represents the response from getting a user's upcoming time off."""

    time_off_events: list[UpcomingTimeOff]

    #Schema: ClassVar[Type[Schema]] = Schema


@tool
def get_upcoming_time_off(
    user_id: str, start_date: str, end_date: str, time_off_types: list[str]
) -> UpcomingTimeOffResponse:
    """
    Retrieves the user's upcoming time off details from SAP SuccessFactors.

    """

    # Check for date format
    try:
        datetime.datetime.strptime(start_date, "%Y-%m-%d")
        datetime.datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return {"error": "Invalid date format. Please use YYYY-MM-DD."}

    time_off_events=[]

    for  time_type in time_off_types:
    # Hard-coded values for testing
        if time_type == "ABSENCE":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Vacation",
                    start_date="2024-01-01",
                    end_date="2024-01-05",
                    start_time=None,
                    end_time=None,
                    duration=5,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="ABSENCE",
                    status_formatted=None,
                    absence_duration_category=None,
                ) )
        elif time_type == "PUBLIC_HOLIDAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="New Year's Day",
                    start_date="2024-01-01",
                    end_date="2024-01-01",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))
            time_off_events.append (UpcomingTimeOff(
                    title="Christmas Day",
                    start_date="2024-12-25",
                    end_date="2024-12-25",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

        elif time_type == "NON_WORKING_DAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Weekend",
                    start_date="2024-01-06",
                    end_date="2024-01-07",
                    start_time=None,
                    end_time=None,
                    duration=2,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="NON_WORKING_DAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

    
    if (len(time_off_events)==0):
        return {"status_code": 400, "error": "Invalid values for time off types"}
    else:
        return UpcomingTimeOffResponse(time_off_events)
'''

## Define custom configurations

Specify the LLM and the provider with the required credentials

In [3]:
from altk.build_time.test_case_generation_toolkit.core.config import TestCaseGenConfig
from altk.core.llm import GenerationMode, get_llm
import os

WatsonXAIClient = get_llm("watsonx")
client = WatsonXAIClient(
    model_id="mistralai/mistral-medium-2505",
    api_key=os.getenv("WX_API_KEY"),
    project_id=os.getenv("WX_PROJECT_ID"),
    url=os.getenv("WX_URL", "https://us-south.ml.cloud.ibm.com"),
)

config = TestCaseGenConfig(
    llm_client=client,
    gen_mode=GenerationMode.CHAT,
    max_nl_utterances=5,
    max_testcases=5,
    clean_nl_utterances=True,  # clean special characters if present in the generated nl utterance
    negative_test_cases=True,
)

### Tool Test Case Generation

In [4]:
from altk.core.toolkit import AgentPhase
from altk.build_time.test_case_generation_toolkit.core.toolkit import (
    TestCaseGenBuildInput,
)
from altk.build_time.test_case_generation_toolkit.utils.test_case_generation import (
    NLTestCaseGenComponent,
)
import pprint

test_case_gen_input = TestCaseGenBuildInput(python_tool_str=python_tool)
test_case_generation_toolkit = NLTestCaseGenComponent()
result = test_case_generation_toolkit.process(
    data=test_case_gen_input, config=config, phase=AgentPhase.BUILDTIME
)
pprint.pprint(result.nl_test_cases)

{'Number_of_mandatory_parameters_covered': 4,
 'Number_of_optional_parameters_covered': 0,
 'Number_of_positive_test_scenarios': 2,
 'Skill_operation': 'python -get_upcoming_time_off , function - '
                    'get_upcoming_time_off',
 'Test_scenarios': [{'id': 'TC_1',
                     'input': 'user_id((str)) := emp_789012 \n'
                              'start_date((str)) := 2023-11-01 \n'
                              'end_date((str)) := 2023-11-10 \n'
                              "time_off_types((list[str])) := ['vacation', "
                              "'sick_leave'] \n",
                     'input_parameters': {'end_date': ['2023-11-10'],
                                          'start_date': ['2023-11-01'],
                                          'time_off_types': [['vacation',
                                                              'sick_leave']],
                                          'user_id': ['emp_789012']},
                     'mandatory_par